# UFoE Full 실험 — Google Colab

환경 설정(OpenMM, ESMFold) 후 Phase 2 전체 파이프라인 실행 및 결과 시각화.

## 1. 환경 설정

In [ ]:
# 프로젝트 클론 (또는 Drive 마운트 후 경로 지정)
!git clone https://github.com/YOUR_ORG/ufoe_protein_experiment.git 2>/dev/null || true
%cd ufoe_protein_experiment 2>/dev/null || %cd /content/drive/MyDrive/ufoe_protein_experiment

In [ ]:
# 의존성 설치
!pip install -q biopython numpy scipy pandas requests

# OpenMM (Full MD용, 선택)
!pip install -q openmm 2>/dev/null || echo "OpenMM 설치 실패 — Mock 모드로 진행"

# ESMFold는 API 사용 시 추가 설치 불필요 (requests만 사용)

## 2. 후보 서열 생성 (선택)

In [ ]:
!python3 generate_candidates.py --n 10 --type B --output candidates/ --esmfold api

## 3. Full 파이프라인 실행

In [ ]:
# 기준 파이프라인(Claude Code)이 있으면 해당 경로에서 실행
import sys, os
sys.path.insert(0, '.')
claude_dir = '../claude code/ufoe_protein_experiment'
if os.path.isdir(claude_dir):
    os.chdir(claude_dir)
    sys.path.insert(0, claude_dir)
from src.pipeline.pipeline import Phase2Pipeline, PipelineConfig
config = PipelineConfig(output_dir='output/colab_run', run_mode='quick', esmfold_mode='api')
pipeline = Phase2Pipeline(config)
results = pipeline.run_full()
pipeline.save_results(results)
Phase2Pipeline.print_summary(results)

## 4. 결과 시각화

In [ ]:
import matplotlib.pyplot as plt
if hasattr(results, 'ufoe_2a_report') and results.ufoe_2a_report and results.control_random_report:
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    ax[0].bar(['UFoE 2a', 'Control'], [results.ufoe_2a_report.success_rate, results.control_random_report.success_rate])
    ax[0].set_ylabel('접힘 성공률')
    if results.elimination_ufoe_2a and results.elimination_control:
        ax[1].bar(['UFoE 2a', 'Control'], [results.elimination_ufoe_2a.self_consistency_ratio, results.elimination_control.self_consistency_ratio])
        ax[1].set_ylabel('K/M 자기일관성')
    plt.tight_layout()
    plt.show()
elif isinstance(results, dict):
    fig, ax = plt.subplots(1, 1, figsize=(5, 4))
    ax.bar(['UFoE'], [results.get('fold_success_rate', 0)])
    ax.set_ylabel('접힘 성공률')
    plt.show()